In [ ]:
import boto3
import json
import time

# 입력받은 버킷의 파일 조회
def bucket_element(name):
    my_bucket = boto3.resource('s3').Bucket(name)
    result = []
    
    for element in my_bucket.objects.all():
        result.append(element.key)
    
    return result

# 로컬에 있는 파일을 rekognition celebrity 분석
def recognition_celebrities(photo):
    client = boto3.client('rekognition')
    name = ''
    
    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})
   
    for celebrity in response['CelebrityFaces']:
        name = celebrity['Name']
    
    return name 
    
# 프로젝트 - 영상내에 존재하는 유명인사 찾기
def recognition_celebrities_video(bucket, key):
    client=boto3.client('rekognition')
    response = client.start_celebrity_recognition(
        Video={'S3Object': {'Bucket': bucket, 
                            'Name': key}})
    startJobId=response['JobId']
    
    print('Start JobId: ' + startJobId)
    print()
    
    return startJobId
    
# 위 함수를 실행하고 반복적으로 실행. 넘겨받은 ID를 가지고 계속 get하다보면 어느 순간
# 'IN_PROGRESS'가 'SUCCEEDED'로 바뀜
def recognition_celebrities_video_get(Id):
    client = boto3.client('rekognition')
    response = client.get_celebrity_recognition(JobId=Id)
    
    return response

# 영상 분석 결과에서 유명인사 리스트를 가져온다.
def listing_celebrity(response, list_celebrity):
    for celebrityRecognition in response['Celebrities']:
        cel_name = str(celebrityRecognition['Celebrity']['Name'])
        if not cel_name in list_celebrity:
            list_celebrity.append(cel_name)

# 평점이 가장 높은 작품을 사용자에게 추천함
def maximum_index(grade_list):
    m = 0
    index = 0
    result = 0
    
    for grade in grade_list:
        if m < grade:
            m = grade
            result = index
        index = index + 1
    
    return result

# main함수
def main():
    bucket_name = 'netflix-video-storage' #aws s3 버켓 이름
    bucket_files = [] # 버켓 내 있는 영상 파일명
    list_celebrity = [] # 버켓 내 있는 모든 영상 분석 결과
    index_list = [] # 유명인사가 출연한 작품의 index리스트 -> bucket_files의 리스트
    celebrity_pro = [] # 유명인사가 출연한 작품을 저장할 리스트
    grade_list = [] # 출연작들의 평점을 저장할 리스트
    user_input = 'user_input1.jpg' #내가 찾고 싶은 유명인사
    celebrity = '' #내가 준 사진이 어떤 유명인사인지...
    
    bucket_files = bucket_element(bucket_name)
    length = len(bucket_files)
    for i in bucket_files:
        list_celebrity.append(list())
    
    celebrity = recognition_celebrities(user_input)
    print("user input result = ", celebrity)
    
    i = 0
    for file in bucket_files:
        print("filename : ", file)
        Id = recognition_celebrities_video(bucket_name, file)
        response = recognition_celebrities_video_get(Id)
        
        print("Analyzing Video...(Id : {})".format(Id))
        while response['JobStatus'] != 'SUCCEEDED':
            time.sleep(10)
            response = recognition_celebrities_video_get(Id)
        
        print("Analysis complete!!!(Id : {})".format(Id))
        print("Save Celebrities...")
        listing_celebrity(response, list_celebrity[i])
        i = i + 1
        print()
    
    i = 0
    for celebrities in list_celebrity:
        if celebrity in celebrities:
            index_list.append(i)
        i = i + 1
        
    for index in index_list:
        file = bucket_files[index]
        file = (file.split('.'))[0]
        celebrity_pro.append(file)
        
    f = open('data.txt', 'r', encoding='utf-8')
    i = 0
    while True:
        line = f.readline().strip('\n')
        if not line: 
            break
        
        if i == len(celebrity_pro):
            break
        
        if line != celebrity_pro[i]:
            line = f.readline().strip('\n')
            continue
        
        grade = f.readline().strip('\n')
        grade_list.append(float(grade))
        i = i + 1
                      
    f.close()
    
    i = 0
    index = maximum_index(grade_list)
    print("Appeared Program List of {}".format(celebrity))
    for pro in celebrity_pro:
        print("{}. {}".format(i+1, pro))
        i + 1
        
    print()
    print("recommendation : {}".format(celebrity_pro[index]))
    
main()